In [22]:
import pandas as pd
import os
import torch
import sqlite3
from sqlite3 import Error

def show_tables(conn):
    """ show all tables in the database """
    cur = conn.cursor()
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    cur.execute(query)
    result = cur.fetchall()
    return [result[i][0] for i in range(len(result))]

def drop_temporary_tables(conn):
    """ drop temporary tables in the database """
    cur = conn.cursor()
    temporary_table_names = [table for table in show_tables(conn) if 'temporary' in table]
    for temporary_table_name in temporary_table_names:
        cur.execute(f"DROP TABLE IF EXISTS {temporary_table_name};")
    conn.commit()
    
def print_query(query,conn):
    cur = conn.cursor()
    cur.execute(query)
    result = cur.fetchall()
    for row in result:
        print(row)

def get_recording_start_stop(conn):
    """
    author: Andrew Smith
    date: 17 April 2025
    description: Get recording start and stop times from the zdb database 
    table internal_property. The table internal_property gets created the 
    first time someone opens a
    Ponemah experiment in NeuroScore. So, the recording start stop times are
    somewhere in the Ponemah files.

    future: get recording start stop directly from Ponemah files to try and
    avoid zdb
    """
    """ get recording start and stop times """
    cur = conn.cursor()
    query = "SELECT value FROM internal_property WHERE key='RecordingStart'"
    cur.execute(query)
    result = cur.fetchall()
    recording_start = int(result[0][0])
    query = "SELECT value FROM internal_property WHERE key='RecordingStop'"
    cur.execute(query)
    result = cur.fetchall()
    recording_stop = int(result[0][0])
    return recording_start, recording_stop

import torch
import os
import torch
from torch import nn
from torch.nn.functional import relu
from mne.io import read_raw_edf
from tqdm import tqdm
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

class ResidualBlock(nn.Module):
    def __init__(self,in_feature_maps,out_feature_maps,n_features) -> None:
        super().__init__()
        self.c1 = nn.Conv1d(in_feature_maps,out_feature_maps,kernel_size=8,padding='same',bias=False)
        self.bn1 = nn.LayerNorm((out_feature_maps,n_features),elementwise_affine=False)

        self.c2 = nn.Conv1d(out_feature_maps,out_feature_maps,kernel_size=5,padding='same',bias=False)
        self.bn2 = nn.LayerNorm((out_feature_maps,n_features),elementwise_affine=False)

        self.c3 = nn.Conv1d(out_feature_maps,out_feature_maps,kernel_size=3,padding='same',bias=False)
        self.bn3 = nn.LayerNorm((out_feature_maps,n_features),elementwise_affine=False)

        self.c4 = nn.Conv1d(in_feature_maps,out_feature_maps,1,padding='same',bias=False)
        self.bn4 = nn.LayerNorm((out_feature_maps,n_features),elementwise_affine=False)

    def forward(self,x):
        identity = x
        x = self.c1(x)
        x = self.bn1(x)
        x = relu(x)

        x = self.c2(x)
        x = self.bn2(x)
        x = relu(x)

        x = self.c3(x)
        x = self.bn3(x)
        x = relu(x)

        identity = self.c4(identity)
        identity = self.bn4(identity)

        x = x+identity
        x = relu(x)
        
        return x
    
class Frodo(nn.Module):
    def __init__(self,n_features) -> None:
        super().__init__()
        self.n_features = n_features
        self.block1 = ResidualBlock(1,8,n_features)
        self.block2 = ResidualBlock(8,16,n_features)
        self.block3 = ResidualBlock(16,16,n_features)

        self.gap = nn.AvgPool1d(kernel_size=n_features)
        self.fc1 = nn.Linear(in_features=16,out_features=3)
    def forward(self,x,classification=True):
        x = x.view(-1,1,self.n_features)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.gap(x)
        if(classification):
            x = self.fc1(x.squeeze())
            return x
        else:
            return x.squeeze()
        
class Gandalf(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.encoder = Frodo(n_features=5000)
        self.lstm = nn.LSTM(16,32,bidirectional=True)
        self.fc1 = nn.Linear(64,3)
    def forward(self,x_2d,classification=True):
        x_2d = x_2d.view(-1,9,1,5000)
        x = []
        for t in range(x_2d.size(1)):
            xi = self.encoder(x_2d[:,t,:,:],classification=False)
            x.append(xi.unsqueeze(0))
        x = torch.cat(x)
        out,_ = self.lstm(x)
        if(classification):
            x = self.fc1(out[-1])
        else:
            x = out[-1]
        return x
    
class EEGDataset(torch.utils.data.Dataset):
    def __init__(self,X):
        self.len = len(X)
        self.X = torch.cat([torch.zeros(4,5000),X,torch.zeros(4,5000)])

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.X[idx:idx+9].flatten()
    
def score_recording(edf_path,model_path,eeg_ch_name,device,zdb_path):
    if not os.path.exists(model_path):
        raise FileExistsError(model_path)

    model = Gandalf()
    model.load_state_dict(torch.load(model_path,map_location='cpu'))

    raw = read_raw_edf(input_fname=edf_path)
    data = raw.get_data(picks=eeg_ch_name)
    eeg = torch.from_numpy(data[0]).float()
    eeg = eeg.view(-1,5000)

    model.eval()
    model.to(device)

    with torch.no_grad():
        dataloader = DataLoader(EEGDataset(eeg),batch_size=32)
        logits = torch.cat([model(Xi.to(device)).cpu() for Xi in tqdm(dataloader)])
        y_pred = logits.softmax(dim=1).argmax(axis=1)

    pd.DataFrame(y_pred,columns=['y_pred']).to_csv(f"{zdb_path.replace('.zdb','.y_pred')}",index=False)
    # pd.DataFrame(logits).to_csv(f"{edf_path.replace('.edf','.logits')}",index=False)

In [24]:
import os
recordings_dir = f'data'
recording_ids = sorted([file.replace('.zdb','') for file in os.listdir(recordings_dir) if file.endswith('.zdb')])
eeg_ch_name = "EEG 1"
device = 'cuda'
model_path = 'gandalf.pt'

for recording_id in recording_ids:
    zdb_path = f'{recordings_dir}/{recording_id}.zdb'
    edf_path = f'{recordings_dir}/{recording_id.split(".")[1]}.edf'
    score_recording(edf_path=edf_path,model_path=model_path,eeg_ch_name=eeg_ch_name,device=device,zdb_path=zdb_path)

Extracting EDF parameters from /home/andrew/aurora-sleep-staging/data/25-Feb-AAV-2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/tmp/ipykernel_593868/3555921427.py:162: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path,map_location='cpu'))
  0%|          | 0/18

Extracting EDF parameters from /home/andrew/aurora-sleep-staging/data/25-Feb-AAV-3.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


100%|██████████| 1899/1899 [01:12<00:00, 26.10it/s]


In [30]:
import os
recordings_dir = f'data'
recording_ids = sorted([file.replace('.zdb','') for file in os.listdir(recordings_dir) if file.endswith('.zdb')])
eeg_ch_name = "EEG 1"
device = 'cuda'
model_path = 'gandalf.pt'

for recording_id in recording_ids:
    zdb_path = f'{recordings_dir}/{recording_id}.zdb'
    y_pred = pd.read_csv(f"{zdb_path.replace('.zdb','.y_pred')}")
    y_pred = pd.DataFrame(y_pred).astype('str')
    y_pred.loc[y_pred['y_pred'] == '0','y_pred'] = 'Sleep-Paradoxical'
    y_pred.loc[y_pred['y_pred'] == '1','y_pred'] = 'Sleep-SWS'
    y_pred.loc[y_pred['y_pred'] == '2','y_pred'] = 'Sleep-Wake'

    try:
        conn = sqlite3.connect(zdb_path)
    except Error as e:
        print(e)
    
    recording_start,recording_stop = get_recording_start_stop(conn)
    length_ns = recording_stop - recording_start # ns
    length_s = length_ns * 1e-7 # s
    hh = length_s // 3600
    mm = (length_s % 3600) // 60
    ss = ((length_s % 3600) % 60)

    print("duration:",hh,"hours",mm,"minutes",ss,"seconds")
    print("duration seconds:",length_s)
    print("recording start:",recording_start)
    print("recording stop:",recording_stop)

    tables = show_tables(conn)

    print(tables)

    cur = conn.cursor()

    if 'scoring_revision' not in tables:
        query = "CREATE TABLE scoring_revision (id INTEGER PRIMARY KEY, name TEXT, is_deleted INTEGER(1), tags TEXT, version INTEGER(8), owner TEXT, date_created INTEGER(8));"
        cur.execute(query)

    if 'scoring_marker' not in tables:
        query = "CREATE TABLE scoring_marker (id INTEGER PRIMARY KEY, starts_at INTEGER(8), ends_at INTEGER(8), notes TEXT, type TEXT, location TEXT, is_deleted INTEGER(1), key_id INTEGER);"
        cur.execute(query)

    if 'scoring_key' not in tables:
        query = "CREATE TABLE scoring_key (id INTEGER PRIMARY KEY, date_created INTEGER(8), name TEXT, owner TEXT, type TEXT);"
        cur.execute(query)

    if 'scoring_revision_to_key' not in tables:
        query = "CREATE TABLE scoring_revision_to_key (revision_id INTEGER(8), key_id INTEGER(8));"
        cur.execute(query)

    conn.commit()

    cur = conn.cursor()
    cur.execute("INSERT INTO scoring_key (type) VALUES ('Automatic');")
    conn.commit()
    key_id = cur.lastrowid
    
    cur = conn.cursor()
    cur.execute("INSERT INTO scoring_revision (name, is_deleted, version) VALUES ('Gandalf', 0, 0);")
    conn.commit()
    scoring_revision_id = cur.lastrowid

    cur = conn.cursor()
    cur.execute(f"INSERT INTO scoring_revision_to_key (revision_id, key_id) VALUES ({scoring_revision_id}, {key_id});")
    conn.commit()

    tables = show_tables(conn)

    print(tables)

    epoch_start_time = int(int(recording_start * 1e-8) * 1e8) # decaseconds
    epoch_stop_time = epoch_start_time + int(1e8)

    for score in y_pred.values.flatten():
        query = f"""
            INSERT INTO scoring_marker 
            (starts_at, ends_at, notes, type, location, is_deleted, key_id)
            VALUES 
            ({epoch_start_time}, {epoch_stop_time}, '', '{score}', '', 0, {key_id});
            """
        
        epoch_start_time = epoch_stop_time
        epoch_stop_time = epoch_start_time + int(1e8)
        
        cur = conn.cursor()
        cur.execute(query)
        
    conn.commit()
    conn.close()

duration: 168.0 hours 44.0 minutes 52.0 seconds
duration seconds: 607492.0
recording start: 638760838920000000
recording stop: 638766913840000000
['internal_property', 'logging_log_entry', 'workspace_workspace', 'scoring_revision', 'scoring_marker', 'scoring_key', 'scoring_revision_to_key']
['internal_property', 'logging_log_entry', 'workspace_workspace', 'scoring_revision', 'scoring_marker', 'scoring_key', 'scoring_revision_to_key']
duration: 168.0 hours 44.0 minutes 52.0 seconds
duration seconds: 607492.0
recording start: 638760838920000000
recording stop: 638766913840000000
['internal_property', 'logging_log_entry', 'workspace_workspace']
['internal_property', 'logging_log_entry', 'workspace_workspace', 'scoring_revision', 'scoring_marker', 'scoring_key', 'scoring_revision_to_key']


In [31]:
os.system(f'tar -czvf data.tar.gz data/*.zdb')

data/25-Feb-AAV.25-Feb-AAV-2.20250225173812.zdb
data/25-Feb-AAV.25-Feb-AAV-3.20250225173812.zdb


0